In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))  

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
from models.mobilenetv3 import MobileNetV3Extractor
from models.lstm_attention import BiLSTMWithAttention
from preprocessing.dataset import SignLanguageDataset

In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
    # resize to the standard input size
    A.Resize(224, 224),  
    # A.RandomCrop(200, 200),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2),
    #A.Cutout(num_holes=2, max_h_size=40, max_w_size=40, fill_value=0),
     # resize it back to 224 to prevent inconsistent dimensions after enhancement
    A.Resize(112, 112), 
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

In [4]:
train_root = "../data/processed_pt/train"
#class_names = sorted(os.listdir(train_root))
class_names = sorted([
    d for d in os.listdir(train_root)
    if os.path.isdir(os.path.join(train_root, d)) and not d.startswith(".")
]) 

label_map = {name: idx for idx, name in enumerate(class_names)}

train_dataset = SignLanguageDataset(
    root_dir=train_root,
    label_map=label_map,
    transform=None 
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

In [5]:
class FullSLRModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = MobileNetV3Extractor()
        self.temporal_model = BiLSTMWithAttention(
            input_dim=960, hidden_dim=256, num_classes=num_classes
        )

    def forward(self, x):  # x: [B, T, C, H, W]
        features = self.feature_extractor(x)          # [B, T, 960]
        logits, _ = self.temporal_model(features)     # [B, num_classes]
        return logits

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FullSLRModel(num_classes=len(label_map)).to(device)

# Label smoothing is helpful for generalization and reduces overfitting
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/opt/anaconda3/envs/slr-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/slr-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = MobileNetV3Extractor().to(device)
sequence_model = BiLSTMWithAttention(input_dim=960, hidden_dim=256, num_classes=len(label_map)).to(device)

model = nn.Sequential(feature_extractor, sequence_model).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [19]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for videos, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training"):
        videos, labels = videos.to(device), labels.to(device)  # [B, T, C, H, W]

        # 
        features = feature_extractor(videos)        # [B, T, D]
        outputs, attn = sequence_model(features)    # [B, num_classes]

        # 
        loss = criterion(outputs, labels)

        # 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 
        total_loss += loss.item()
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # 
    epoch_loss = total_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    print(f"[Train] Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")


Epoch [1/3] Training:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch [1/3] Training: 100%|██████████| 2/2 [00:31<00:00, 15.66s/it]


[Train] Epoch 1/3 | Loss: 5.6665 | Acc: 0.00%


Epoch [2/3] Training: 100%|██████████| 2/2 [00:31<00:00, 15.93s/it]


[Train] Epoch 2/3 | Loss: 5.4269 | Acc: 100.00%


Epoch [3/3] Training: 100%|██████████| 2/2 [00:23<00:00, 11.68s/it]

[Train] Epoch 3/3 | Loss: 5.1600 | Acc: 100.00%


In [8]:
os.makedirs("../checkpoints", exist_ok=True)
torch.save(model.state_dict(), "../checkpoints/mobilenetv3_lstm_aug_smooth.pth")